# Star alleles

## Table of contents

1. [Non-rsID records](#Non-rsID-records)
2. [Genotype/allele annotations](#Genotype/allele-annotations)
3. [Allele definition tables](#Allele-definition-tables)
4. [Comparison with PharmVar](#Comparison-with-PharmVar)
6. [Summary and questions](#Summary)

In [1]:
import pandas as pd
import requests

from opentargets_pharmgkb.pandas_utils import read_tsv_to_df

In [2]:
work_dir = '/home/april/projects/opentargets/pharmgkb/star-alleles'

In [ ]:
# Rerun to refresh data
!cd {work_dir}
!wget -q https://api.pharmgkb.org/v1/download/file/data/clinicalAnnotations.zip
!unzip -qj clinicalAnnotations.zip "*.tsv" -d {work_dir}
!rm clinicalAnnotations.zip

## Non-rsID records

[Top of page](#Table-of-contents)

In [3]:
annotations_df = read_tsv_to_df(f'{work_dir}/clinical_annotations.tsv')
alleles_df = read_tsv_to_df(f'{work_dir}/clinical_ann_alleles.tsv')

In [4]:
len(annotations_df)

5101

In [5]:
no_rs_annotations = annotations_df[~annotations_df['Variant/Haplotypes'].str.contains('rs')]

In [6]:
len(no_rs_annotations)

596

In [7]:
# Check names to see if there's anything truly bizarre
names = no_rs_annotations['Variant/Haplotypes'].unique()

In [57]:
# Note that the "variant/haplotype name" is a listing of which alleles are annotated in the specific record
names[:50]

array(['HLA-B*15:02',
       'CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41',
       'CYP2D6*1, CYP2D6*1xN, CYP2D6*2xN, CYP2D6*4, CYP2D6*5',
       'CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*2xN',
       'CYP2D6*1, CYP2D6*3, CYP2D6*4, CYP2D6*4xN, CYP2D6*5, CYP2D6*6',
       'CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D6*10x2, CYP2D6*11, CYP2D6*17, CYP2D6*21, CYP2D6*36, CYP2D6*41',
       'UGT1A3*1, UGT1A3*2, UGT1A3*3', 'HLA-B*55:01',
       'CYP2C19*1, CYP2C19*17',
       'NAT2*4, NAT2*5, NAT2*6, NAT2*7, NAT2*12, NAT2*13',
       'CYP3A5*1, CYP3A5*3', 'CYP2C9*1, CYP2C9*3',
       'CYP2C19*1, CYP2C19*2, CYP2C19*3', 'UGT1A1*1, UGT1A1*28',
       'CYP2B6*1, CYP2B6*6', 'NUDT15*1, NUDT15*4, NUDT15*5, NUDT15*6',
       'NUDT15*1, NUDT15*6', 'CYP2D6*1, CYP2D6*10', 'UGT1A1*1, UGT1A1*6',
       'CYP2C9*1, CYP2C9*2, CYP2C9*3', 'HLA-B*48:01',
       'CYP2C19*1, CYP2C19*2, CYP2C19*17', 'HLA-B*15:12',
  

In [9]:
# Not necessarily an important distinction, but just to check...
star_allele_names = [n for n in names if '*' in n]
no_star_names = [n for n in names if '*' not in n]

In [10]:
no_star_names

['G6PD A- 202A_376G, G6PD B (reference)',
 'GSTT1 non-null, GSTT1 null',
 'GSTM1 non-null, GSTM1 null',
 'G6PD A- 202A_376G, G6PD B (reference), G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham',
 'SLC6A4 HTTLPR long form (L allele), SLC6A4 HTTLPR short form (S allele)',
 'G6PD B (reference), G6PD Mediterranean Haplotype',
 'G6PD B (reference), G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham',
 'G6PD B (reference), G6PD Canton, Taiwan-Hakka, Gifu-like, Agrigento-like',
 'G6PD B (reference), G6PD Mediterranean Haplotype, G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham',
 'G6PD A- 202A_376G, G6PD B (reference), G6PD Mediterranean Haplotype, G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham',
 'G6PD A- 202A_376G']

No star allele observations:
* [G6PD](https://www.pharmgkb.org/gene/PA28469/haplotype) seems well-defined though the naming is idiosyncratic (e.g. is it safe to just comma-split these strings?)
  * may be a bit clearer in the alleles tables, e.g. [here](https://www.pharmgkb.org/clinicalAnnotation/1183621000)
* [GSTT1](https://www.pharmgkb.org/gene/PA183/haplotype), [GSTM1](https://www.pharmgkb.org/gene/PA182/haplotype) null/non-null are just absence or presence of the entire gene, if this naming convention is standard we can work with it
* [SLC6A4](https://www.pharmgkb.org/gene/PA312/haplotype) seems to be just... special

Note we can clearly get affected genes for all of these alleles though, from PGKB directly.

In [61]:
# Confirming there are no missing genes in any of these
no_rs_annotations['Gene'].isna().any()

False

## Genotype/allele annotations

[Top of page](#Table-of-contents)

In [65]:
pd.set_option('display.max_colwidth', None)

In [66]:
joined_df = alleles_df.merge(no_rs_annotations, on='Clinical Annotation ID')

In [67]:
# Remove some columns to make things easier to read...
joined_df = joined_df[['Clinical Annotation ID', 'Genotype/Allele', 'Annotation Text',
       'Allele Function', 'Variant/Haplotypes', 'Gene', 'Level of Evidence',
       'Phenotype Category', 'Drug(s)', 'Phenotype(s)']]

In [72]:
# https://www.pharmgkb.org/clinicalAnnotation/1451259580
joined_df[joined_df['Clinical Annotation ID'] == '1451259580']

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s)
1,1451259580,*1,The CYP2D6*1 allele is assigned as a normal function allele by CPIC. Patients carrying the CYP2D6*1 allele in combination with alleles that result in a normal metabolizer phenotype who are treated with amitriptyline may have decreased likelihood of side effects as compared to patients with a combination of alleles that result in intermediate or poor metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,Normal function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
2,1451259580,*1xN,The CYP2D6*1xN alleles (*1x2 and *1x≥3) have been assigned as increased function alleles by CPIC. Patients carrying the CYP2D6*1xN allele in combination with alleles that result in a normal metabolizer phenotype who are treated with amitriptyline may have decreased likelihood of side effects as compared to patients with a combination of alleles that result in intermediate or poor metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,Increased function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
3,1451259580,*2,The CYP2D6*2 allele is assigned as a normal function allele by CPIC. Patients carrying the CYP2D6*2 allele in combination with alleles that result in a normal metabolizer phenotype who are treated with amitriptyline may have decreased likelihood of side effects as compared to patients with a combination of alleles that result in intermediate or poor metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,Normal function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
4,1451259580,*3,The CYP2D6*3 allele is assigned as a no function allele by CPIC. Patients carrying the CYP2D6*3 allele in combination with with alleles that result in intermediate or poor metabolizer phenotype who are treated with amitriptyline may have increased likelihood of side effects as compared to patients with alleles that result in a normal metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,No function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
5,1451259580,*4,The CYP2D6*4 allele is assigned as a no function allele by CPIC. Patients carrying the CYP2D6*4 allele in combination with with alleles that result in intermediate or poor metabolizer phenotype who are treated with amitriptyline may have increased likelihood of side effects as compared to patients with alleles that result in a normal metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,No function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
6,1451259580,*5,The CYP2D6*5 allele is assigned as a no function allele by CPIC. Patients carrying the CYP2D6*5 allele in combination with with alleles that result in intermediate or poor metabolizer phenotype who are treated with amitriptyline may have increased likelihood of side effects as compared to patients with alleles that result in a normal metabolizer phenotype. Other genetic and clinical factors may also influence response to amitriptyline.,No function,"CYP2D6*1, CYP2D6*1xN, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*5, CYP2D6*6, CYP2D6*10, CYP2D6*41",CYP2D6,1A,Toxicity,amitriptyline,Depressive Disorder
7,1451259580,*6,The CYP2D6*6 allele is assigned as a n

In [73]:
# https://www.pharmgkb.org/clinicalAnnotation/1448427588
joined_df[joined_df['Clinical Annotation ID'] == '1448427588']

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s)
1551,1448427588,non-null/non-null,Patients with the non-null/non-null genotype may have a decreased risk for neutropenia when treated with clozapine as compared to patients with the null/null genotype. Other genetic and clinical factors may also influence neutropenia risk.,NaN,"GSTT1 non-null, GSTT1 null",GSTT1,3,Toxicity,clozapine,NaN
1552,1448427588,null/non-null,Patients with the null/non-null genotype may have a decreased risk for neutropenia when treated with clozapine as compared to patients with the null/null genotype. Other genetic and clinical factors may also influence neutropenia risk.,NaN,"GSTT1 non-null, GSTT1 null",GSTT1,3,Toxicity,clozapine,NaN
1553,1448427588,null/null,Patients with the null/null genotype may have an increased risk for neutropenia when treated with clozapine as compared to patients with the null/non-null or non-null/non-null genotype. Other genetic and clinical factors may also influence neutropenia risk.,NaN,"GSTT1 non-null, GSTT1 null",GSTT1,3,Toxicity,clozapine,NaN


In [74]:
# https://www.pharmgkb.org/clinicalAnnotation/981419263
joined_df[joined_df['Clinical Annotation ID'] == '981419263']

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s)
498,981419263,*15:02,"Patients with one or two copies of the HLA-B*15:02 allele may have an increased risk of Severe Cutaneous Adverse Reactions when treated with carbamazepine as compared to patients with no HLA-B*15:02 alleles or negative for the HLA-B*15:02 test. However, conflicting evidence has been reported. Other genetic and clinical factors may also influence risk of carbamazepine-induced adverse reactions.",Presence,"HLA-B*15:02, HLA-B*15:11",HLA-B,1A,Toxicity,carbamazepine,"drug reaction with eosinophilia and systemic symptoms;Epidermal Necrolysis, Toxic;Maculopapular Exanthema;severe cutaneous adverse reactions;Stevens-Johnson Syndrome"
499,981419263,*15:11,"Patients with one or two copies of the HLA-B*15:11 allele may have an increased risk of Severe Cutaneous Adverse Reactions, such as Stevens-Johnson Syndrome and Toxic Epidermal Necrolysis, when treated with carbamazepine as compared to patients with no HLA-B*15:11 alleles or negative for the HLA-B*15:11 test. However, conflicting evidence has been reported. Other genetic and clinical factors may also influence risk of carbamazepine-induced adverse reactions.",NaN,"HLA-B*15:02, HLA-B*15:11",HLA-B,1A,Toxicity,carbamazepine,"drug reaction with eosinophilia and systemic symptoms;Epidermal Necrolysis, Toxic;Maculopapular Exanthema;severe cutaneous adverse reactions;Stevens-Johnson Syndrome"


In [75]:
# https://www.pharmgkb.org/clinicalAnnotation/1183621000
joined_df[joined_df['Clinical Annotation ID'] == '1183621000']

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s)
524,1183621000,A- 202A_376G,"Patients with one X-chromosome and the A- 202A_376G allele who are treated with rasburicase may have an increased risk of methemoglobinemia and/or hemolysis as compared to patients with the reference B allele (non-deficient, class IV). Patients with two X-chromosomes and the A- 202A_376G allele in combination with another deficient class I-III allele who are treated with rasburicase may have an increased risk of methemoglobinemia and/or hemolysis as compared to patients with two copies of the reference B allele (non-deficient, class IV). Patients with two X-chromosomes and the A- 202A_376G allele in combination with a non-deficient allele who are treated with rasburicase have an unknown risk of methemoglobinemia and/or hemolysis as compared to patients with two copies of the reference B allele (non-deficient, class IV). Other genetic and clinical factors may also influence risk of drug-induced hemolysis.",III/Deficient,"G6PD A- 202A_376G, G6PD B (reference), G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham",G6PD,1A,Toxicity,rasburicase,Hemolysis;Methemoglobinemia
525,1183621000,B (reference),"Patients with one X-chromosome and the reference B (reference) allele (non-deficient, class IV) who are treated with rasburicase may have a decreased risk of methemoglobinemia and/or hemolysis as compared to patients with a deficient class I-III allele. Patients with two X-chromosomes and two copies of the reference B allele (non-deficient, class IV) who are treated with rasburicase may have a decreased risk of methemoglobinemia and/or hemolysis as compared to patients with a deficient class I-III allele. Patients with two X-chromosomes, one copy of the reference B allele (non-deficient, class IV) and one deficient class I-III allele who are treated with rasburicase have an unknown risk of methemoglobinemia and/or hemolysis as compared to patients with two copies of the reference B allele (non-deficient, class IV). Other genetic and clinical factors may also influence risk of drug-induced hemolysis.",IV/Normal,"G6PD A- 202A_376G, G6PD B (reference), G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham",G6PD,1A,Toxicity,rasburicase,Hemolysis;Methemoglobinemia
526,1183621000,"Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham","Patients with one X-chromosome and the Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham allele (rs5030868 allele A) who are treated with rasburicase may have an increased risk of methemoglobinemia and/or hemolysis as compared to patients with the reference B allele (non-deficient, class IV)(rs5030868 allele G). Patients with two X-chromosomes and the Mediterranean, Dallas, Panama' Sassari, Cagliari, Birmingham variant (rs5030868 allele A) in combination with another deficient class I-III allele who are treated with rasburicase may have an increased risk of methemoglobinemia and/or hemolysis as compared to patients with two copies of the reference B allele (non-deficient, class IV)(rs5030868 allele G). Patients with two X-chromosomes and the Mediterranean, Dallas, Panama' Sassari, Cagliari, Birmingham variant (rs5030868 allele A) in combination with a non-deficient allele who are treated with rasburicase have an unknown risk of methemoglobinemia and/or hemolysis as compared to patients with two copies of the reference B allele (non-deficient, class IV). Other genetic and clinical factors may also influence risk of drug-induced hemolysis.",II/Deficient,"G6PD A- 202A_376G, G6PD B (reference), G6PD Mediterranean, Dallas, Panama, Sassari, Cagliari, Birmingham",G6PD,1A,Toxicity,rasburicase,Hemolysis;Methemoglobinemia


Notes:
* These are mostly allele-level annotations, though some are genotype-level (as opposed to rsID records which are nearly all genotype-level)
    * Genotype-specific information is sometimes buried in the annotation text for each allele...
* `*1xN` means `N` copies of the `*1` version of the gene

## Allele definition tables

[Top of page](#Table-of-contents)

In [14]:
# Try to automatically get PGKB spreadsheet definitions
allele_definition_url = 'https://api.pharmgkb.org/v1/download/file/attachment/{gene}_allele_definition_table.xlsx'

In [62]:
genes = no_rs_annotations['Gene'].unique()

In [63]:
genes

array(['HLA-B', 'CYP2D6', 'UGT1A3', 'CYP2C19', 'NAT2', 'CYP3A5', 'CYP2C9',
       'UGT1A1', 'CYP2B6', 'NUDT15', 'CYP2C8', 'CYP3A4', 'HLA-A', 'G6PD',
       'UGT2B15', 'SLCO1B1', 'GSTT1', 'GSTM1', 'TPMT', 'SLC6A4', 'HLA-C',
       'HLA-DRB1', 'HLA-DQB1', 'HLA-DPB1', 'CYP3A7', 'CYP2A6', 'HLA-DRB3',
       'CYP1A2', 'UGT1A6', 'CYP2E1', 'UGT1A7', 'HLA-DQA1', 'UGT1A4',
       'CYP1A1', 'CYP4F2'], dtype=object)

In [17]:
allele_def_tables = {}
for gene in genes:
    try:
        allele_def_tables[gene] = pd.read_excel(allele_definition_url.format(gene=gene), storage_options={'User-Agent': 'Mozilla/5.0'})
    except Exception as e:
        print(f'Error for {gene}: {e}')

/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Error for UGT1A3: HTTP Error 404: 


/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Error for NAT2: HTTP Error 404: 


/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply o

...

/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Error for CYP1A2: HTTP Error 404: 
Error for UGT1A6: HTTP Error 404: 
Error for CYP2E1: HTTP Error 404: 
Error for UGT1A7: HTTP Error 404: 


/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Error for UGT1A4: HTTP Error 404: 
Error for CYP1A1: HTTP Error 404: 


/home/april/projects/opentargets-pharmgkb/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
# Genes with allele tables
pharmgkb_genes = set(allele_def_tables.keys())
pharmgkb_genes

{'CYP2A6',
 'CYP2B6',
 'CYP2C19',
 'CYP2C8',
 'CYP2C9',
 'CYP2D6',
 'CYP3A4',
 'CYP3A5',
 'CYP4F2',
 'G6PD',
 'HLA-A',
 'HLA-B',
 'HLA-C',
 'HLA-DPB1',
 'HLA-DQA1',
 'HLA-DQB1',
 'HLA-DRB1',
 'HLA-DRB3',
 'NUDT15',
 'SLCO1B1',
 'TPMT',
 'UGT1A1'}

In [38]:
no_allele_def_table_genes = set(genes) - pharmgkb_genes

In [39]:
no_allele_def_table_genes

{'CYP1A1',
 'CYP1A2',
 'CYP2E1',
 'CYP3A7',
 'GSTM1',
 'GSTT1',
 'NAT2',
 'SLC6A4',
 'UGT1A3',
 'UGT1A4',
 'UGT1A6',
 'UGT1A7',
 'UGT2B15'}

Checked a few of this list and they indeed don't have definition tables in PharmGKB, categories I see:
* Refer to another resource: [some (but not all) CYP](https://www.pharmgkb.org/gene/PA129), [NAT](https://www.pharmgkb.org/gene/PA18/haplotype), [UGT](https://www.pharmgkb.org/gene/PA37179/haplotype)
* Null/non-null: [GSTT1](https://www.pharmgkb.org/gene/PA183/haplotype), [GSTM1](https://www.pharmgkb.org/gene/PA182/haplotype)
* Special: [SLC6A4](https://www.pharmgkb.org/gene/PA312/haplotype)

For now we'll skip these and look at those with allele definition tables (covers about 90% of no-RS records in PGKB).

In [25]:
# What do we lose if we skip these?
len(no_rs_annotations[no_rs_annotations['Gene'].isin(no_allele_def_table_genes)])

53

Note that the allele definition tables vary in informativeness, so just because one is present doesn't mean we can necessarily use it.
* More informative example: [CYP2D6](https://docs.google.com/spreadsheets/d/1tIovgq2w7FXv6g2ASiQd5EtYjxEbSzYrCRpg9dIlrEY/edit?usp=sharing)
* Less informative example: [HLA-A](https://docs.google.com/spreadsheets/d/1Wz0F74sdY-hG0LJP1Rg5ePsZddVQ1pZrHDySOYgrOhI/edit?usp=sharing)

Understanding the allele definition table:
* First few rows give various definitions of variants: protein/chromosome/gene-level HGVS, and rsID if present
* Each subsequent row gives what alleles are present for each of these variants for a particular named allele
    * In theory should be able to use the "Genotype/Allele" column from the clinical allele annotations to index into this table
* The final column is "structural variation" and contains text describing the nature of the variant, e.g. `CYP2D7::CYP2D6 hybrid gene`
* Missing values = reference? Or is e.g. *1/first row the reference? If so what does missing value mean?

## Comparison with PharmVar

[Top of page](#Table-of-contents)

In [43]:
# Compare with what we would get from PharmVar
pharmvar_url = 'https://www.pharmvar.org/api-service/alleles?exclude-sub-alleles=false&include-reference-variants=false&include-retired-alleles=false&include-retired-reference-sequences=false'
response = requests.get(pharmvar_url)
pharmvar_data = response.json()

In [44]:
# 1 per allele
len(pharmvar_data)

1945

In [45]:
pharmvar_data[0]

{'geneSymbol': 'CYP2C9',
 'alleleName': 'CYP2C9*49.001',
 'pvId': 'PV00001',
 'legacyLabel': 'CYP2C9*49',
 'coreAllele': 'CYP2C9*49',
 'evidenceLevel': 'L',
 'description': None,
 'function': 'uncertain function',
 'activeInd': True,
 'references': [{'citation': 'Dai et al. 2013',
   'url': 'http://www.ncbi.nlm.nih.gov/pubmed/23400009'}],
 'variants': [{'referenceSequence': 'NG_008385.2',
   'referenceLocation': 'ATG Start',
   'referenceCollections': ['RefSeqGene'],
   'hgvs': 'NG_008385.2:g.15972A>G',
   'rsId': None,
   'impact': 'I222V',
   'variantFrequency': [],
   'url': 'https://www.pharmvar.org/variant/13610',
   'variantId': '49',
   'position': 'NG_008385.2:g.10447A>G'},
  {'referenceSequence': 'NM_000771.4',
   'referenceLocation': 'ATG Start',
   'referenceCollections': ['RefSeqTranscript'],
   'hgvs': 'NM_000771.4:c.664A>G',
   'rsId': None,
   'impact': 'I222V',
   'variantFrequency': [],
   'url': 'https://www.pharmvar.org/variant/13765',
   'variantId': '49',
   'posit

In [46]:
pharmvar_genes = {d['geneSymbol'] for d in pharmvar_data}

In [48]:
pharmgkb_genes - pharmvar_genes

{'G6PD',
 'HLA-A',
 'HLA-B',
 'HLA-C',
 'HLA-DPB1',
 'HLA-DQA1',
 'HLA-DQB1',
 'HLA-DRB1',
 'HLA-DRB3',
 'TPMT',
 'UGT1A1'}

In [49]:
pharmvar_genes - pharmgkb_genes

{'CYP2A13', 'DPYD'}

In [50]:
len(no_rs_annotations[no_rs_annotations['Gene'].isin(pharmvar_genes - pharmgkb_genes)])

0

Conclusion from this is that PharmVar probably has less information than PharmGKB; though most of the genes covered by PGKB and not by PV are "uninformative" tables, there are at least 2 exceptions (G6PD and UGT1A1). In contrast PV genes not covered by PGKB are not present in PGKB data.

I haven't compared the actual content of the PV vs. PGKB data but I'm assuming it's similar since it's sourced directly from PV.

Implementation-wise, PV does have the advantage in that it has an actual API with JSON responses rather than spreadsheets.

## Summary

* Non-rsID containing records represent 596 / 5101 = 12% of the clinical annotations
* Affected gene is easy to get for all named alleles - we can rely on the "Gene" column in PGKB data
* Annotations tend to be per-allele rather than per-genotype
* Most records have an allele definition table from PGKB that we can download
    * 53 / 596 = 8.9% do not
    * Not all of these tables contain specific variants - see [CYP2D6](https://docs.google.com/spreadsheets/d/1tIovgq2w7FXv6g2ASiQd5EtYjxEbSzYrCRpg9dIlrEY/edit?usp=sharing) vs. [HLA-A](https://docs.google.com/spreadsheets/d/1Wz0F74sdY-hG0LJP1Rg5ePsZddVQ1pZrHDySOYgrOhI/edit?usp=sharing)
* PGKB alleles tables mostly (but not entirely) come from PharmVar

### Questions

* Identifier for these?
    * PGKB basically uses a list of haplotypes being annotated as the "variant" identifier in their annotations table
    * Note that our PGx schema uses genotype IDs not variant IDs
* Do we want to resolve named alleles to variants, and if so how to convey this information?
* Are we interested in functional consequences or is affected gene enough?
    * Could try using [Haplosaurus](https://www.ensembl.org/info/docs/tools/vep/haplo/index.html) - use allele definition table & annotated allele to create VCF file input

[Top of page](#Table-of-contents)